In [14]:
import time
import pandas as pd

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Indica la ruta del chromedriver
service = Service('/Users/jobandtalent/Downloads/chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=service)

# Abre la página de Google Maps
driver.get("https://www.google.com/maps/place/Cervecer%C3%ADa+Sierra/@40.4400515,-3.7104325,15z/data=!4m8!3m7!1s0xd42285d9fe6f033:0xe27e4de86263432a!8m2!3d40.4342037!4d-3.7104844!9m1!1b1!16s%2Fg%2F11b6d43yfv?entry=ttu&g_ep=EgoyMDI0MDkxNi4wIKXMDSoASAFQAw%3D%3D")


In [8]:
# Espera a que el cuadro de reseñas esté disponible
wait = WebDriverWait(driver, 10)
reviews_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')))

raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.m6QErb.DxyBCb.kA9KIf.dS8AEf.XiKgde')

print(raw_html_reviews[0].text)

raw_html_stars_resumme = raw_html_reviews[0].find_elements(By.CSS_SELECTOR, 'tr.BHOKXe')

ratings = []
reviews_counts = []

for star in raw_html_stars_resumme:
    rating_text = star.get_attribute('aria-label')
    rating_parts = rating_text.split(',')  # Dividimos la información por coma
    stars = rating_parts[0].split()[0]  # Extraemos el número de estrellas
    num_reviews = rating_parts[1].strip().split()[0].replace('.', '')  # Extraemos el número de reseñas y eliminamos el punto
    ratings.append(int(stars))
    reviews_counts.append(int(num_reviews))

stars_resumme = pd.DataFrame({'stars': ratings, 'reviews': reviews_counts})
display(stars_resumme)

No se pudo extraer una de las reseñas


### Basic

In [30]:
raw_html_reviews_list = raw_html_reviews[0].find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')

# Creamos listas para almacenar los datos
reviewers = []
ratings = []
review_texts = []
review_dates = []

# Recorremos cada reseña
for review in raw_html_reviews_list:
    try:
        # Extraer el nombre del autor
        reviewer = review.find_element(By.CSS_SELECTOR, 'div.d4r55').text
        # Extraer el texto de la reseña
        review_text = review.find_element(By.CSS_SELECTOR, 'span.wiI7pd').text
        # Extraer la calificación (estrellas)
        rating = review.find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label')
        # Extraer la fecha de la reseña
        review_date = review.find_element(By.CSS_SELECTOR, 'span.rsqaWe').text
        
        # Añadir a las listas
        reviewers.append(reviewer)
        review_texts.append(review_text)
        ratings.append(rating)
        review_dates.append(review_date)
        
    except Exception as e:
        print(f"Error al extraer una reseña: {str(e)}")

data = {'Autor': reviewers, 'Calificación': ratings, 'Reseña': review_texts, 'Fecha': review_dates}
df = pd.DataFrame(data)
df

### All

In [44]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Indica la ruta del chromedriver
service = Service('/Users/jobandtalent/Downloads/chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=service)

driver.get("https://www.google.com/maps/place/Cervecer%C3%ADa+Sierra/@40.4400515,-3.7104325,15z/data=!4m8!3m7!1s0xd42285d9fe6f033:0xe27e4de86263432a!8m2!3d40.4342037!4d-3.7104844!9m1!1b1!16s%2Fg%2F11b6d43yfv?entry=ttu&g_ep=EgoyMDI0MDkxNi4wIKXMDSoASAFQAw%3D%3D")


# Espera hasta que el contenedor de reseñas esté disponible
wait = WebDriverWait(driver, 10)
reviews_container = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.m6QErb.XiKgde')))

# Inicializamos variables para almacenar los datos
reviewers = []
ratings = []
review_texts = []
review_dates = []
prices = []
food_ratings = []
service_ratings = []
ambiance_ratings = []
recommended_dishes = []

# Scroll loop
SCROLL_PAUSE_TIME = 2
while True:
    # Extraemos todas las reseñas visibles
    raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')

    # Para cada reseña visible
    for review in raw_html_reviews:
        try:
            # Intentamos hacer clic en el botón "Ver más" si está presente
            try:
                more_buttons = review.find_elements(By.CSS_SELECTOR, 'button.w8nwRe')
                for button in more_buttons:
                    if button.is_displayed():  # Solo hacemos clic si es visible
                        button.click()
                        time.sleep(1)  # Espera un momento para cargar la reseña completa
            except Exception as e:
                print(f"No se encontró un botón 'Ver más' en esta reseña: {str(e)}")

            # Extraer el nombre del autor
            reviewer = review.find_element(By.CSS_SELECTOR, 'div.d4r55').text
            # Extraer el texto de la reseña
            review_text = review.find_element(By.CSS_SELECTOR, 'span.wiI7pd').text
            # Extraer la calificación (estrellas)
            rating = review.find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label')
            # Extraer la fecha de la reseña
            review_date = review.find_element(By.CSS_SELECTOR, 'span.rsqaWe').text

            # Intentar extraer la información adicional después de pulsar el botón "más"
            try:
                price = review.find_element(By.XPATH, '//span[text()="Precio por persona"]/following-sibling::span').text
                food_rating = review.find_element(By.XPATH, '//span[text()="Comida:"]/following-sibling::span').text
                service_rating = review.find_element(By.XPATH, '//span[text()="Servicio:"]/following-sibling::span').text
                ambiance_rating = review.find_element(By.XPATH, '//span[text()="Ambiente:"]/following-sibling::span').text
                recommended_dish = review.find_element(By.XPATH, '//span[text()="Platos recomendados"]/following-sibling::span').text
            except:
                # Si no se encuentran los datos adicionales, almacenamos valores vacíos
                price = "N/A"
                food_rating = "N/A"
                service_rating = "N/A"
                ambiance_rating = "N/A"
                recommended_dish = "N/A"

            # Añadir a las listas
            reviewers.append(reviewer)
            review_texts.append(review_text)
            ratings.append(rating)
            review_dates.append(review_date)
            prices.append(price)
            food_ratings.append(food_rating)
            service_ratings.append(service_rating)
            ambiance_ratings.append(ambiance_rating)
            recommended_dishes.append(recommended_dish)

        except Exception as e:
            print(f"Error al extraer una reseña: {str(e)}")

    # Hacer scroll hacia abajo
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", reviews_container)
    time.sleep(SCROLL_PAUSE_TIME)

    # Verificar si ya no se cargan más reseñas nuevas
    new_raw_html_reviews = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')
    if len(new_raw_html_reviews) == len(raw_html_reviews):  # Si no aparecen más reseñas, paramos
        break

# Crear un DataFrame a partir de las listas
data = {
    'Autor': reviewers,
    'Calificación': ratings,
    'Reseña': review_texts,
    'Fecha': review_dates,
    'Precio': prices,
    'Comida': food_ratings,
    'Servicio': service_ratings,
    'Ambiente': ambiance_ratings,
    'Platos recomendados': recommended_dishes
}

df = pd.DataFrame(data)

# Mostrar el DataFrame
print(df)


                        Autor Calificación  \
0                      LO TIZ  5 estrellas   
1                Blvck Kinder  5 estrellas   
2  Arkaitz fernandez sanabria  5 estrellas   
3   ANGELA “LocuraTodoLoCura”  5 estrellas   
4           Claudia M. Simal.  5 estrellas   
5                       David  5 estrellas   
6                       César  4 estrellas   
7                Dxvid Martos  5 estrellas   

                                              Reseña            Fecha Precio  \
0  Fantástico ambiente. Camareros rápidos y amabl...  Hace una semana    N/A   
1  El Bar Sierra en Madrid es un verdadero tesoro...     Hace 2 meses    N/A   
2  Tapas absolutamente de locos. Por apenas 3.30 ...     Hace 5 meses    N/A   
3  La leche!! Este sitio es el mejor con diferenc...  Hace una semana    N/A   
4  Por el precio nos gustó mucho, los tamaños de ...     Hace 6 meses    N/A   
5  En la zona de Chamberí en Madrid. La estación ...     Hace 2 meses    N/A   
6  Os cuento según mi opi

In [45]:
df.iloc[0].Reseña

'Fantástico ambiente. Camareros rápidos y amables. Unas tapas espectaculares. Totalmente recomendable. El ambiente me ha encantado. Volveremos!!😍😍'

In [46]:
df

,Autor,Calificación,Reseña,Fecha,Precio,Comida,Servicio,Ambiente,Platos recomendados
0,LO TIZ,5 estrellas,Fantástico ambiente. Camareros rápidos y amabl...,Hace una semana,N/A,N/A,N/A,N/A,N/A
1,Blvck Kinder,5 estrellas,El Bar Sierra en Madrid es un verdadero tesoro...,Hace 2 meses,N/A,N/A,N/A,N/A,N/A
2,Arkaitz fernandez sanabria,5 estrellas,Tapas absolutamente de locos. Por apenas 3.30 ...,Hace 5 meses,N/A,N/A,N/A,N/A,N/A
3,ANGELA “LocuraTodoLoCura”,5 estrellas,La leche!! Este sitio es el mejor con diferenc...,Hace una semana,N/A,N/A,N/A,N/A,N/A
4,Claudia M. Simal.,5 estrellas,"Por el precio nos gustó mucho, los tamaños de ...",Hace 6 meses,N/A,N/A,N/A,N/A,N/A
5,David,5 estrellas,En la zona de Chamberí en Madrid. La estación ...,Hace 2 meses,N/A,N/A,N/A,N/A,N/A
6,César,4 estrellas,Os cuento según mi opinión:\n\nContras:\nSitio...,Hace 3 meses,N/A,N/A,N/A,N/A,N/A
7,Dxvid Martos,5 estrellas,"El camarero mu majo, el baño limpio y las tapa...",Hace un mes,N/A,N/A,N/A,N/A,N/A


In [42]:
df.head(2).Reseña.values

array(['Fantástico ambiente. Camareros rápidos y amables. Unas tapas espectaculares. Totalmente recomendable. El ambiente me ha encantado. Volveremos!!😍😍',
       'El Bar Sierra en Madrid es un verdadero tesoro para los amantes de la gastronomía. Este bar se destaca por sus tapas gigantes, que no solo impresionan por su tamaño, sino también por su sabor y calidad.\nEl servicio en el Bar Sierra es perfecto.'],
      dtype=object)

In [52]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# Función para hacer clic en todos los botones "Más" que expanden las reseñas
def click_expand_buttons(driver):
    expand_buttons = driver.find_elements(By.CSS_SELECTOR, 'button.w8nwRe.kyuRq')
    for button in expand_buttons:
        try:
            driver.execute_script("arguments[0].click();", button)
        except Exception as e:
            print(f"Error al hacer clic en el botón: {e}")

# Función para extraer información de cada reseña, incluyendo precio, comida, servicio, ambiente y platos recomendados
def extract_reviews_info(driver):
    reviews_elements = driver.find_elements(By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')
    reviews_data = []
    
    for review in reviews_elements:
        try:
            # Hacer clic en el botón "Más" para expandir reseña completa
            click_expand_buttons(driver)
            
            reviewer_name = review.get_attribute('aria-label')
            stars = review.find_element(By.CSS_SELECTOR, 'span.kvMYJc').get_attribute('aria-label')
            review_date = review.find_element(By.CSS_SELECTOR, 'span.rsqaWe').text
            review_text = review.find_element(By.CSS_SELECTOR, 'span.wiI7pd').text

            # Extraer precio, comida, servicio, ambiente y platos recomendados
            price_per_person = review.find_element(By.CSS_SELECTOR, "span[aria-label*='€']").text
            food_rating = review.find_element(By.XPATH, ".//span[contains(text(), 'Comida')]/b").text if 'Comida' in review.text else None
            service_rating = review.find_element(By.XPATH, ".//span[contains(text(), 'Servicio')]/b").text if 'Servicio' in review.text else None
            atmosphere_rating = review.find_element(By.XPATH, ".//span[contains(text(), 'Ambiente')]/b").text if 'Ambiente' in review.text else None
            recommended_dishes = review.find_element(By.XPATH, ".//span[contains(text(), 'Platos recomendados')]/following-sibling::span").text if 'Platos recomendados' in review.text else None
            
            review_element = {
                'reviewer_name': reviewer_name,
                'stars': stars,
                'date': review_date,
                'review_text': review_text,
                'price_per_person': price_per_person,
                'food_rating': food_rating,
                'service_rating': service_rating,
                'atmosphere_rating': atmosphere_rating,
                'recommended_dishes': recommended_dishes
            }
            print(review_element)
            reviews_data.append(review_element)
        except Exception as e:
            print(f"Error al extraer reseña: {e}")
    
    # Convertir la información en un DataFrame para su visualización y análisis
    df_reviews = pd.DataFrame(reviews_data)
    display(df_reviews)
    return df_reviews

# Indica la ruta del chromedriver
service = Service('/Users/jobandtalent/Downloads/chromedriver-mac-arm64/chromedriver')
driver = webdriver.Chrome(service=service)

driver.get("https://www.google.com/maps/place/Cervecer%C3%ADa+Sierra/@40.4400515,-3.7104325,15z/data=!4m8!3m7!1s0xd42285d9fe6f033:0xe27e4de86263432a!8m2!3d40.4342037!4d-3.7104844!9m1!1b1!16s%2Fg%2F11b6d43yfv?entry=ttu&g_ep=EgoyMDI0MDkxNi4wIKXMDSoASAFQAw%3D%3D")

# Esperar a que las reseñas carguen
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.jftiEf.fontBodyMedium')))
extract_reviews_info(driver)


Error al extraer reseña: Message: no such element: Unable to locate element: {"method":"xpath","selector":".//span[contains(text(), 'Comida')]/b"}
  (Session info: chrome=129.0.6668.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102fffed4 cxxbridge1$str$ptr + 1906348
1   chromedriver                        0x0000000102ff8344 cxxbridge1$str$ptr + 1874716
2   chromedriver                        0x0000000102c0c264 cxxbridge1$string$len + 89492
3   chromedriver                        0x0000000102c50514 cxxbridge1$string$len + 368708
4   chromedriver                        0x0000000102c46ad8 cxxbridge1$string$len + 329224
5   chromedriver                        0x0000000102c8a7d4 cxxbridge1$string$len + 606980
6   chromedriver                        0x0000000102c45134 cxxbridge1$string$len + 322660
7   chromedriver               

""


""


In [51]:
reviews_elements

NameError: name 'reviews_elements' is not defined